<a href="https://colab.research.google.com/github/coming1024/US-Election-Study-2023/blob/main/US_Election_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive ## 挂载云盘
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import os
os.chdir('/content/drive/MyDrive/')
!ls

 benchmark1	   'CVPR2020_GHUM_Paper (1).gdoc'   hashtag_joebiden.csv
'Colab Notebooks'   CVPR2020_GHUM_Paper.gdoc


In [26]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

#For Preprocessing
import re    # RegEx for removing non-letter characters
import nltk  # natural language processing
nltk.download("stopwords")
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# For Building the model
from sklearn.model_selection import train_test_split
import tensorflow as tf
import seaborn as sns

#For data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

pd.options.plotting.backend = "plotly"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [18]:
df=pd.read_csv('./hashtag_joebiden.csv')
df

<ipython-input-18-41658c3c539e>:1: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('./hashtag_joebiden.csv')


,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:18,1.31652929585929e+18,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,Twitter for iPad,809904438.0,Cheri A. 🇺🇸,Biloximeemaw,"Locked and loaded Meemaw. Love God, my family ...",...,6628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.517827283
2,2020-10-15 00:00:20,1.3165293050069524e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3494182277.0,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
3,2020-10-15 00:00:21,1.3165293080815575e+18,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,Twitter for iPhone,8.242596012018524e+17,Michelle Ferg,MichelleFerg4,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.553481849
4,2020-10-15 00:00:22,1.316529312741253e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032806955356545e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059904,2020-11-08 23:59:38,1.3255888248362353e+18,Stop laying @CNN !\n#Paris and #London dont gi...,0.0,0.0,Twitter for iPhone,387361991.0,flaquet,flaquet1,NaN,...,27.0,PARIS,48.856697,2.351462,Paris,France,Europe,Ile-de-France,IDF,2020-11-09 18:32:45.613547
1059905,2020-11-08 23:59:38,1.325588825654096e+18,Ωχ ελπίζω να μη μας βγει σαν τους οπαδούς του...,0.0,0.0,Twitter for Android,403281875.0,οχι άλλο κάρβουνο 🇬🇷🗣🗣🗣,anapodoi,ακραία καιρικά φαινόμενα... ζήσαμε και απόψε,...,772.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.947617
1059906,2020-11-08 23:59:41,1.3255888381237576e+18,L'OTAN va sortir de sa léthargie et redevenir ...,48.0,14.0,Twitter for Android,781918265400492032.0,🇫🇷 Alt-Droite (matricule 6921) ✝️ 🇬🇷 🇮🇹 🇦🇲,CtrlAltDroite,Fils de mineur. Libertés - Identité - Solidari...,...,15806.0,France,46.603354,1.888334,NaN,France,Europe,NaN,NaN,2020-11-09 18:32:45.627335
1059907,2020-11-08 23:59:52,1.3255888813460152e+18,🌎\n\n“#congiuntifuoriregione”\n\n‘Sono felice ...,1.0,1.0,Twitter for iPhone,529331509.0,Angelo Tani,AngeloTani,nato a casa dei nonni,...,5974.0,🌎,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-09 18:32:45.599846


In [19]:
df=df[['tweet','likes','retweet_count']][0:10000]
df

,tweet,likes,retweet_count
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0
1,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0
2,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0
3,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0
4,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0
...,...,...,...
9995,Reasons to Vote for Joe! https://t.co/HfjAesDn...,1.0,0.0
9996,मोदी के खुले समर्थन के बावजूद 22 भारतीय-अमेरिक...,9.0,3.0
9997,Smoking gun emails from the Biden’s #JoeBide...,0.0,0.0
9998,#Twitter &amp; #Facebook action over Joe Biden...,1.0,1.0


In [20]:
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words
    
cleantext=[]
for item in tqdm(df['tweet']):
    words=tweet_to_words(str(item))
    cleantext+=[words]
df['cleantext']=cleantext
df

100%|██████████| 10000/10000 [00:28<00:00, 351.71it/s]


,tweet,likes,retweet_count,cleantext
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,"[elecciones2020, en, florida, joebiden, dice, ..."
1,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,"[hunterbiden, hunterbidenemail, joebiden, joeb..."
2,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,"[islandgirlprv, bradbeauregardj, meidastouch, ..."
3,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,"[chrislongview, watch, set, dvr, let, give, bo..."
4,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,"[censorship, hunterbiden, biden, bidenemail, b..."
...,...,...,...,...
9995,Reasons to Vote for Joe! https://t.co/HfjAesDn...,1.0,0.0,"[reason, vote, joe, http, co, hfjaesdnyq, bide..."
9996,मोदी के खुले समर्थन के बावजूद 22 भारतीय-अमेरिक...,9.0,3.0,"[22, joebiden, realdonaldtrump, uselection2020..."
9997,Smoking gun emails from the Biden’s #JoeBide...,0.0,0.0,"[smoke, gun, email, biden, joebiden, election2..."
9998,#Twitter &amp; #Facebook action over Joe Biden...,1.0,1.0,"[twitter, amp, facebook, action, joe, biden, j..."


In [21]:
def unlist(list):
    words=''
    for item in list:
        words+=item+' '
    return words

In [22]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    df["vader_neg"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["vader_neu"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["vader_pos"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["vader_comp"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    df['cleantext2'] = df[label].apply(lambda x: unlist(x))
    return df

In [28]:
df2 = compute_vader_scores(df,'cleantext')
df2

,tweet,likes,retweet_count,cleantext,vader_neg,vader_neu,vader_pos,vader_comp,cleantext2
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,"[elecciones2020, en, florida, joebiden, dice, ...",0.000,1.000,0.0,0.0000,elecciones2020 en florida joebiden dice que do...
1,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,"[hunterbiden, hunterbidenemail, joebiden, joeb...",0.000,1.000,0.0,0.0000,hunterbiden hunterbidenemail joebiden joebiden...
2,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,"[islandgirlprv, bradbeauregardj, meidastouch, ...",0.000,1.000,0.0,0.0000,islandgirlprv bradbeauregardj meidastouch bide...
3,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,"[chrislongview, watch, set, dvr, let, give, bo...",0.000,1.000,0.0,0.0000,chrislongview watch set dvr let give bonu rate...
4,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,"[censorship, hunterbiden, biden, bidenemail, b...",0.000,1.000,0.0,0.0000,censorship hunterbiden biden bidenemail bidene...
...,...,...,...,...,...,...,...,...,...
9995,Reasons to Vote for Joe! https://t.co/HfjAesDn...,1.0,0.0,"[reason, vote, joe, http, co, hfjaesdnyq, bide...",0.000,1.000,0.0,0.0000,reason vote joe http co hfjaesdnyq biden2020 j...
9996,मोदी के खुले समर्थन के बावजूद 22 भारतीय-अमेरिक...,9.0,3.0,"[22, joebiden, realdonaldtrump, uselection2020...",0.000,1.000,0.0,0.0000,22 joebiden realdonaldtrump uselection2020 joe...
9997,Smoking gun emails from the Biden’s #JoeBide...,0.0,0.0,"[smoke, gun, email, biden, joebiden, election2...",0.179,0.821,0.0,-0.3400,smoke gun email biden joebiden election2020 co...
9998,#Twitter &amp; #Facebook action over Joe Biden...,1.0,1.0,"[twitter, amp, facebook, action, joe, biden, j...",0.060,0.940,0.0,-0.1027,twitter amp facebook action joe biden joebiden...
